# Introduction

In [1]:
# import libraries
import model_param
import sys
import os
import math
import numpy as np
np.set_printoptions(precision=3)

# Input file

In [2]:
data_file = ''

def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    data_file = 'Calib_1.str'
else:
    if ( len(sys.argv) < 2 ):
        print('\nSupply WinProp Raydata file in ASCII format.\n\t\tUsage ', 
              sys.argv[0], ': Raydata_file', '\n')
        exit()
    data_file = str(sys.argv[1])

if ( not os.path.isfile(data_file) ):
    print("\nPlease check the filename", data_file, " and try again.\n")
    exit()

# Parser

This section is a module for parsing incoming ascii file (in str format)

It returns a numerical matrix with 5 columns as shown below:


| Delay(ns)  | Strength(dBuV/m)  |  X-coordinate |  Y-coordinate |  Z-coordinate |
|------------|-------------------|---------------|---------------|---------------|
| 2349.565   |      58.71        |     396.00    |     580.00    |    515.46     |
|  926.787   |      74.92        |     736.00    |     980.00    |    515.06     |
|  ......    |      .....        |     .....     |     ......    |    ......     |
|  ......    |      .....        |     .....     |     ......    |    ......     |
|  808.820   |      76.04        |     756.00    |     980.00    |    517.05     |

In [3]:
# Description:
#   Parse incoming ascii file (in str format)
#
# returns:
#  numpy matrix with 5 columns as follows:
#
#   Delay    FieldStrength   X-coordinate  Y-coordinate  Z-coordinate
#   [ns]       [dBuV/m]       
#   
#
def parse_input(filename):

    point = []
    dataset = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            columns = line.split()

            # Do not look at lines that are empty, 
            # commented or have less than 4 cols.
            if ( len(columns) == 0 or columns[0] == '*' or len(columns) < 4):
                continue ;

            if ( columns[0] == 'PATH' and len(point)==3 ):
                # Parameters: delay (ns) , power (dBuV/m)
                record = [float(num) for num in columns[1:3] + point[:]]
                dataset += record

            if ( columns[0] == 'POINT' ):
                # Co-ordinates x, y, z
                point = columns[1:4]

    return np.asarray(dataset).reshape(-1,5)

# Prepare regression dataset

In [4]:
# Format: Delay Power X Y Z
dataset = parse_input(data_file)

# Normalize Dataset

In [5]:
# normalize the dataset
data_mean = np.min(dataset, axis=0)*0.99
data_std  = (np.max(dataset, axis=0)-np.min(dataset, axis=0));

def normalize(d, mean, std):
    return (d - mean) / std

def denormalize(d, mean, std):
    return (d * std) + mean

dataset = normalize(dataset, data_mean, data_std)

print('Dataset properties:')
print('Dataset size', dataset.shape)
for i in range(dataset.shape[1]):
    print('Min=', np.min(dataset[:,i]),  'Max=',np.max(dataset[:,i]), 
          'Mean=',np.mean(dataset[:,i]), 'sigma=', np.std(dataset[:,i]))

Dataset properties:
Dataset size (784, 5)
Min= 0.00127944267486 Max= 1.00127944267 Mean= 0.179466008977 sigma= 0.18481504455
Min= 0.00622068024842 Max= 1.00622068025 Mean= 0.62657493406 sigma= 0.144190847335
Min= 0.0312525560868 Max= 1.03125255609 Mean= 0.47824121045 sigma= 0.186312056997
Min= 0.00469889924264 Max= 1.00469889924 Mean= 0.403156520141 sigma= 0.2641023245
Min= 0.0262294279157 Max= 1.02622942792 Mean= 0.586458040488 sigma= 0.190530368099


# Predict Location

In [6]:
def locationModel(d, p):
    x = np.sqrt(model_param.idwx*d*d + model_param.ipwx*(p*p-p) + model_param.ibx)
    y = np.sqrt(model_param.idwy*d*d + model_param.ipwy*(p*p-p) + model_param.iby)
    z = np.sqrt(model_param.idwz*d*d + model_param.ipwz*(p*p-p) + model_param.ibz)

    return x, y, z

def predictLocation(delay, power):
    d = normalize(delay, model_param.data_mean[0], model_param.data_std[0])
    p = normalize(power, model_param.data_mean[1], model_param.data_std[1])
    
    x,y,z = locationModel(d, p)
    
    x = denormalize(x, model_param.data_mean[2], model_param.data_std[2])
    y = denormalize(y, model_param.data_mean[3], model_param.data_std[3])
    z = denormalize(z, model_param.data_mean[4], model_param.data_std[4])

    return x, y, z

# Predict Location

In [7]:
testset = parse_input(data_file)
print ('# Path_Delay Field Strength : [X Y X] => predicted_X predicted_Y predicted_Z')
for i in range(testset.shape[0]):
    print("{:.1f}".format(testset[i,0]), "{:.1f}".format(testset[i,1]), 
          ':', testset[i, 2:], ' => ', '[ {:.3f} {:.3f} {:.3f}]'.format(*predictLocation(testset[i, 0], testset[i, 1])) )

# Path_Delay Field Strength : [X Y X] => predicted_X predicted_Y predicted_Z
2299.2 58.7 : [ 426.381  587.823  572.276]  =>  [ 935.714 1078.691 516.602]
2537.2 53.5 : [ 426.381  587.823  572.276]  =>  [ 966.428 1080.189 516.721]
2325.7 52.1 : [ 426.381  587.823  572.276]  =>  [ 952.421 1084.425 516.696]
2444.7 52.0 : [ 426.381  587.823  572.276]  =>  [ 962.616 1083.130 516.725]
2311.5 53.7 : [ 426.381  587.823  572.276]  =>  [ 946.884 1082.758 516.666]
2246.1 59.9 : [ 395.597  580.393  504.295]  =>  [ 929.783 1078.665 516.581]
2233.8 55.2 : [ 395.597  580.393  504.295]  =>  [ 936.807 1082.086 516.626]
2323.9 50.4 : [ 395.597  580.393  504.295]  =>  [ 957.691 1086.726 516.729]
2525.6 55.3 : [ 395.597  580.393  504.295]  =>  [ 961.191 1078.509 516.691]
2441.8 55.2 : [ 395.597  580.393  504.295]  =>  [ 954.017 1079.603 516.672]
2244.9 62.1 : [ 376.232  610.186  480.097]  =>  [ 927.772 1077.899 516.569]
2350.0 61.0 : [ 376.232  610.186  480.097]  =>  [ 937.312 1076.986 516.597]
2385.1 59.0

1570.3 59.8 : [ 404.28   946.912  554.876]  =>  [ 880.955 1085.134 516.459]
1366.1 65.2 : [ 406.811  870.248  556.563]  =>  [ 866.172 1085.567 516.416]
1425.7 61.1 : [ 418.943  920.811  517.387]  =>  [ 871.015 1085.700 516.431]
1274.6 63.0 : [ 428.965  977.02   520.118]  =>  [ 861.505 1086.222 516.405]
2717.1 55.2 : [ 428.965  977.02   520.118]  =>  [ 978.435 1075.981 516.739]
1344.9 60.4 : [ 420.809  939.092  511.765]  =>  [ 867.283 1086.508 516.425]
2385.1 53.0 : [ 420.809  939.092  511.765]  =>  [ 954.849 1082.680 516.694]
1296.9 57.6 : [ 394.662  920.438  516.636]  =>  [ 868.956 1088.306 516.442]
2668.0 47.9 : [ 394.662  920.438  516.636]  =>  [ 995.667 1086.357 516.868]
1279.6 56.5 : [ 395.94   956.458  574.188]  =>  [ 870.505 1089.279 516.453]
2820.1 56.6 : [ 395.94   956.458  574.188]  =>  [ 985.365 1073.328 516.749]
2611.8 53.0 : [ 395.94   956.458  574.188]  =>  [ 974.425 1079.842 516.748]
2679.3 49.4 : [ 395.94   956.458  574.188]  =>  [ 991.179 1083.736 516.835]
1185.5 54.2 

1134.6 55.7 : [ 420.481  959.497  505.69 ]  =>  [ 865.427 1090.768 516.446]
1599.1 49.2 : [ 420.481  959.497  505.69 ]  =>  [ 911.095 1095.521 516.623]
3284.3 54.5 : [ 420.481  959.497  505.69 ]  =>  [ 1034.814 1067.823 516.909]
2922.4 52.1 : [ 420.481  959.497  505.69 ]  =>  [ 1005.168 1076.489 516.844]
2828.4 51.3 : [ 420.481  959.497  505.69 ]  =>  [ 998.552 1078.879 516.833]
2815.1 44.2 : [ 420.481  959.497  505.69 ]  =>  [ 1023.470 1091.173 517.005]
1221.7 62.1 : [  472.214  1031.76    527.279]  =>  [ 859.356 1086.719 516.402]
1140.5 60.4 : [ 423.773  987.484  511.677]  =>  [ 857.047 1087.721 516.401]
1179.2 65.4 : [ 429.902  995.898  561.975]  =>  [ 856.646 1086.740 516.394]
1102.5 62.1 : [  399.063  1069.828   546.392]  =>  [ 853.719 1087.370 516.389]
1186.9 62.1 : [ 403.047  997.942  496.271]  =>  [ 857.612 1086.905 516.398]
1170.5 53.9 : [ 403.047  997.942  496.271]  =>  [ 871.935 1092.292 516.476]
1658.1 55.1 : [ 403.047  997.942  496.271]  =>  [ 895.495 1087.785 516.523]
173